# Sonar System Object Classification

In this project we apply basic machine learning algorithms to a simple binary classification problem, considering data given by a Sonar System. The aim is to characterize rocks and mines throw the patterns captured by the sonar, each one consists of the measure of energy in a time lapse in different frequencies and varying the angle. We are working within the *MLJ framework* of Julia, which is a compendium of different packages and functions to apply machine learning algorithms.

In [1]:
# Manipulate data
using CSV 
using DataFrames
# Data exploration and visualization
using PrettyPrinting
import Statistics
# Create pseudo-random numbers
using StableRNGs

### Machine Learning Framework ### 
using MLJ 

## 1. Data

### 1.1 Data Exploration

In [2]:
# Loading the data
df = CSV.read("sonar.csv",DataFrame)
first(df,3)

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.228,0.2431,0.3771,0.5598
3,0.01,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598


Note that there are many columns hidden and As we can see, the dataframe does not have a header, so we must built it and load again the data. The first 
columns represent signals (amount of energy) captured by the Sonar, each one in a specific frequency, the last column apparently contains the category labels. 

In [3]:
# Build the header with a list
header = ["Freq$i" for i=1:size(df,2)-1 ]
push!(header,"Label")
# Re Load data and set header
df = CSV.read("sonar.csv",DataFrame,header=header)

# Watch last 7 columns and first 3 rows
first(select(df, 54:61 ),3)

,Freq54,Freq55,Freq56,Freq57,Freq58,Freq59,Freq60,Label
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String1
1,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,R
2,0.0048,0.0094,0.0191,0.014,0.0049,0.0052,0.0044,R
3,0.0095,0.018,0.0244,0.0316,0.0164,0.0095,0.0078,R


In [4]:
print("Dimension of the DataFrame (cols,rows): \t $(size(df)) 
    \nData Type/Format of the first $(size(df,2)-1) columns: \t $(typeof(df[1,1])) 
    \nData Type/Format of the last column: \t \t $(typeof(df[1,end]))")

Dimension of the DataFrame (cols,rows): 	 (208, 61) 
    
Data Type/Format of the first 60 columns: 	 Float64 
    
Data Type/Format of the last column: 	 	 String1

So we have $N=208$ samples, and the dimension of the feature vector is $D=60$. Next, we are obtaining the data description.

In [5]:
describe(select(df, 50:60))

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,Freq50,0.020424,0.0,0.0179,0.0825,0,Float64
2,Freq51,0.0160687,0.0,0.0139,0.1004,0,Float64
3,Freq52,0.0134202,0.0008,0.0114,0.0709,0,Float64
4,Freq53,0.0107091,0.0005,0.00955,0.039,0,Float64
5,Freq54,0.0109409,0.001,0.0093,0.0352,0,Float64
6,Freq55,0.00929038,0.0006,0.0075,0.0447,0,Float64
7,Freq56,0.00822163,0.0004,0.00685,0.0394,0,Float64
8,Freq57,0.00782019,0.0003,0.00595,0.0355,0,Float64
9,Freq58,0.00794904,0.0003,0.0058,0.044,0,Float64


Note that the labels are `M` for mines and `R` for rocks.

To analyse range values in frequency columns in one shot we can: 
- make groups by  `eltype`, obtaining two groups (frequencies and target)
- Describe the mean,min and max over mean, median, max and number of missing value in the first group

In [6]:
describe(groupby(describe(df,:mean,:min,:max,:nmissing,:eltype),:eltype)[1], :mean,:min,:max)

,variable,mean,min,max
,Symbol,Union…,Any,Any
1,variable,,Freq1,Freq9
2,mean,0.281321,0.00650721,0.702155
3,min,0.0189433,0.0,0.0921
4,max,0.655823,0.0352,1.0
5,nmissing,0.0,0,0
6,eltype,,,


We can see that the values of the feature vector for each sample are small floating point numbers between 0 and 1, so it is convenient to store them as `Float64` to avoid roundoff error, also we can be sure that there aren't missing values in the whole dataset.  

Lets take a look on the number of rocks and mines in the dataset. 

In [7]:
num_R = count(i->(i=="R"),df[:,end])
num_M = count(i->(i=="M"),df[:,end])
print("Number of Mines (M): $num_M \nNumber of Rocks (R):  $num_R")

Number of Mines (M): 111 
Number of Rocks (R):  97

As we can see, the distrbution of mines and rocks is not discrete-uniform, this is an important fact we have to consider when spliting the data into train and test sets.

### 1.2 Scientific Types

Lets analyse how the data is interpreted, which means, the Scitype of the variables stored in each column

In [8]:
schema(select(df, 54:61))

┌────────┬────────────┬─────────┐
│ names  │ scitypes   │ types   │
├────────┼────────────┼─────────┤
│ Freq54 │ Continuous │ Float64 │
│ Freq55 │ Continuous │ Float64 │
│ Freq56 │ Continuous │ Float64 │
│ Freq57 │ Continuous │ Float64 │
│ Freq58 │ Continuous │ Float64 │
│ Freq59 │ Continuous │ Float64 │
│ Freq60 │ Continuous │ Float64 │
│ Label  │ Textual    │ String1 │
└────────┴────────────┴─────────┘


The value in each cell of a Frequency column represents an amount of energy, so it is reasonable to let them be continuous Scitypes, but the Label column should be a categorical variable.

In [9]:
df = coerce( df, :Label => OrderedFactor)
schema(select(df,:Label))

┌───────┬──────────────────┬───────────────────────────────────┐
│ names │ scitypes         │ types                             │
├───────┼──────────────────┼───────────────────────────────────┤
│ Label │ OrderedFactor{2} │ CategoricalValue{String1, UInt32} │
└───────┴──────────────────┴───────────────────────────────────┘


Now, the categorical labels identified are

In [10]:
levels(df.Label)

2-element Vector{String1}:
 "M"
 "R"

In the model assesment it is appropiate to interpret a mine as "positive" and a rock as "negative", so we ought to define the order $R < M$ 

In [11]:
levels!(df.Label, ["R","M"])
levels(df.Label)

2-element Vector{String1}:
 "R"
 "M"

One can check the order relation.

In [12]:
print("Category of first sample: $(df.Label[1]) \nCategory of last sample: $(df.Label[end])")

Category of first sample: R 
Category of last sample: M

In [13]:
df.Label[1]<df.Label[end]

true

## 2. Data Set Split

### 2.1 Input-Output split
To make the input-output split (X,y structure) is very easy using the `unpack` function in the MLJ framework.

In [14]:
y, X = unpack(df, ==(:Label));

### 2.2 Train-test split

We are taking 70% of the data for training and the rest for testing. Remember that the amount of rocks and mines is not the same, so we have to guarante that the proportion won't change after the set split. 

The split is done over the data set **indices**, this is the common way in the MLJ framework

In [15]:
# Get rocks indices
rocks_idx = findall(==("R"),y)
# Get mines indices
mines_idx = findall(==("M"),y)

# Set a random seed
rng = StableRNG(0)
# Make a random partition over mines indices 
train_mines , test_mines = partition( mines_idx , 0.7, shuffle=true, rng=rng)
# Make a random partition over rocks indices 
train_rocks , test_rocks = partition( rocks_idx , 0.7, shuffle=true, rng=rng)



# join mines and rocks (indices) for each set
train = [train_mines ; train_rocks]
test = [test_mines ; test_rocks];


## 3. Machine Learning Models

### 3.1 Available Models

After preprocessing our data, especially setting the categorical variables and the other scitypes, it is possible to know which models in the MLJ framework are aproppiate for our data, considering the structure $X,y$. This can be done using the function `models` along with the function `matching`.

In [16]:
for m in models(matching(X, y))
    println(rpad(m.name, 30), "\t ($(m.package_name))")
end

AdaBoostClassifier            	 (ScikitLearn)
AdaBoostStumpClassifier       	 (DecisionTree)
BaggingClassifier             	 (ScikitLearn)
BayesianLDA                   	 (MultivariateStats)
BayesianLDA                   	 (ScikitLearn)
BayesianQDA                   	 (ScikitLearn)
BayesianSubspaceLDA           	 (MultivariateStats)
ConstantClassifier            	 (MLJModels)
DSADDetector                  	 (OutlierDetectionNetworks)
DecisionTreeClassifier        	 (BetaML)
DecisionTreeClassifier        	 (DecisionTree)
DeterministicConstantClassifier	 (MLJModels)
DummyClassifier               	 (ScikitLearn)
ESADDetector                  	 (OutlierDetectionNetworks)
EvoTreeClassifier             	 (EvoTrees)
ExtraTreesClassifier          	 (ScikitLearn)
GaussianNBClassifier          	 (NaiveBayes)
GaussianNBClassifier          	 (ScikitLearn)
GaussianProcessClassifier     	 (ScikitLearn)
GradientBoostingClassifier    	 (ScikitLearn)
KNNClassifier                 	 (NearestNeighborMode

As we can see, most models are implemented outside of the MLJ ecosystem; we therefore have to load models using the `@load` command and specifying the package where it comes from. 

According to the theory we have seen in the signature, the choosen models are: 
- LinearSVC (Support vector machine with linear Kernel)
- SVC (Support vector machine with RBF kernel)
- LogisticClassifier (Logistic regression)
- DecisionTreeClassifier (Ordinary Desicion Tree) 
- KNeighborsClassifier (K-Nearest Neighbors)

### 3.2 Single model Example

In [17]:
# Load the model from the ScikitLearn ecosystem
LogisticClassifier  = @load LogisticClassifier  pkg=ScikitLearn
# Create a model instance with the default hyperparameters
logistic_model = LogisticClassifier()

import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\JSeba\.julia\packages\MLJModels\QEeNR\src\loading.jl:168


LogisticClassifier(
    penalty = "l2",
    dual = false,
    tol = 0.0001,
    C = 1.0,
    fit_intercept = true,
    intercept_scaling = 1.0,
    class_weight = nothing,
    random_state = nothing,
    solver = "lbfgs",
    max_iter = 100,
    multi_class = "auto",
    verbose = 0,
    warm_start = false,
    n_jobs = nothing,
    l1_ratio = nothing)

In MLJ, a **model** is an object that only serves as a container for the hyperparameters of the model. A **machine** is an object wrapping both a model and data and can contain information on the trained model; it does not fit the model by itself. However, it does check that the model is compatible with the scientific type of the data and will warn you otherwise.

In [18]:
# Build a machine
logistic = machine(logistic_model, X, y)
# Train the machine
fit!(logistic, rows=train)

┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase C:\Users\JSeba\.julia\packages\MLJBase\CMT6L\src\machines.jl:464


Machine{LogisticClassifier,…} trained 1 time; caches data
  model: MLJScikitLearnInterface.LogisticClassifier
  args: 
    1:	Source @548 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @966 ⏎ `AbstractVector{OrderedFactor{2}}`


Some Classifiers, as the logistic regression work, with *Probabilistic* predictions, the output value represents the probability of $y^{i}$ being positive (1 or `M`) given $X^{(i)}$. We can get this probabilistic values with the function `prediction`.

In [19]:
# Probabilistic prediction given test input
ŷ_prob = predict(logistic, rows=test);

Also MLJ allows to show this information for one sample in a fancy way by using the  `@show` decorator

In [20]:
@show ŷ_prob[1]

ŷ_prob[1] = UnivariateFinite{OrderedFactor{2}}(R=>0.305, M=>0.695)


         UnivariateFinite{OrderedFactor{2}}     
     ┌                                        ┐ 
   R ┤■■■■■■■■■ 0.3045705477167904              
   M ┤■■■■■■■■■■■■■■■■■■■■ 0.6954294522832096   
     └                                        ┘ 

To get the prediction according to the decision boundary (typically 0.5) one might use `predict_mode` 

In [21]:
# Definite prediction over the test input
ŷ = predict_mode(logistic, rows=test)
ŷ[1:5]

5-element CategoricalArrays.CategoricalArray{String1,1,UInt32}:
 "M"
 "M"
 "M"
 "M"
 "R"

### 3.3 Model Assesment

In classification problems is it always useful to compute the confussion matrix to identify paterns and understand proportions of correct classification and deviations in each class.

In [22]:
confmat(ŷ,y[test])

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      R      │      M      │
├─────────────┼─────────────┼─────────────┤
│      R      │     24      │      7      │
├─────────────┼─────────────┼─────────────┤
│      M      │      5      │     26      │
└─────────────┴─────────────┴─────────────┘


In our case, using the Sonar System we rather to detect all the mines possible and tolerate some falsepositive rocks. So, the most suitable measure is the recall which is computed by 
$$ \text{recall} = \frac{TP}{TP+FN}$$
the recall is also calle as True Positive rate.

In [23]:
logistic_recall = recall(ŷ,y[test]);
print("Recall in logistic regression: $logistic_recall")

Recall in logistic regression: 0.7878787878787878

### 3.4 Model Comparison
Now we will train all the models selected in section 3.1, then evaluate and compare them by the recall metric.

In [24]:
# Load all models 
Linear_SVC = @load LinearSVC verbosity=0
SVC = @load SVC verbosity=0
LogisticClassifier = @load LogisticClassifier pkg=ScikitLearn verbosity=0
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree  verbosity=0
KNeighborsClassifier = @load KNeighborsClassifier  verbosity=0;

In [25]:
# Save all models in a list, so we can easily apply a cycle itering over them.
Models = [ Linear_SVC, SVC, LogisticClassifier, DecisionTreeClassifier, KNeighborsClassifier];

In [26]:
# Create an array to save the scores
scores = []
# Main cycle to train, predict and evaluate every model.
for model in Models
    instance = model()
    mach = machine(instance, X , y)
    fit!(mach, rows=train, verbosity=0)
    
    # Consider If the model has a probabilistic output or not
    try
        ŷ = predict_mode(mach, rows=test)
    catch
        ŷ = predict(mach, rows=test)
    end
    # Evaluating the model perfomance by recall metric
    recall_score = recall(ŷ, y[test])
    println("\t Score: $recall_score \t \t  ----->  $(instance)")
    push!(scores,recall_score)
end


	 Score: 0.8181818181818182 	 	  ----->  LinearSVC
	 Score: 0.8787878787878788 	 	  ----->  SVC
	 Score: 0.7878787878787878 	 	  ----->  LogisticClassifier
	 Score: 0.7272727272727273 	 	  ----->  DecisionTreeClassifier
	 Score: 0.7272727272727273 	 	  ----->  KNeighborsClassifier


### 3.5 Best Model

In [27]:
best_model = Models[findmax(scores)[2]]
pkg, model_name  = split(string(best_model), ".")
print("The best model is $model_name from the $pkg package")

The best model is SVC from the MLJLIBSVMInterface package

The default hyperparameters are 

In [28]:
instance = best_model()

SVC(
    kernel = LIBSVM.Kernel.RadialBasis,
    gamma = 0.0,
    cost = 1.0,
    cachesize = 200.0,
    degree = 3,
    coef0 = 0.0,
    tolerance = 0.001,
    shrinking = true,
    probability = false)

### Fitting analysis

In [29]:
mach = machine(instance, X , y)
fit!(mach, rows=train, verbosity=0)
ŷtrain = predict(mach, rows=train);

The confussion matrix for the trainning set is

In [30]:
display(confmat(ŷtrain,y[train]))

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      R      │      M      │
├─────────────┼─────────────┼─────────────┤
│      R      │     55      │      3      │
├─────────────┼─────────────┼─────────────┤
│      M      │     13      │     75      │
└─────────────┴─────────────┴─────────────┘


In [31]:
accuracy_train = accuracy(ŷtrain,y[train])
recall_train = recall(ŷtrain, y[train])
print("Fitting Accuracy: \t $accuracy_train \nFitting Recall: \t $recall_train")

Fitting Accuracy: 	 0.8904109589041096 
Fitting Recall: 	 0.9615384615384616

We can see that the fitting accuracy is almost 90%, which means that the fitting is not perfect, there are 3 false negatives and 13 false positives, but this is not a big issue. However, it reachs 96% in fitting recall, which is the choosen metric, this might be an explanation of model's succes.

### Performance on the test Data

In [32]:
ŷtest = predict(mach, rows=test);

Confussion Matrix

In [33]:
display(confmat(ŷtest,y[test]))

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      R      │      M      │
├─────────────┼─────────────┼─────────────┤
│      R      │     24      │      4      │
├─────────────┼─────────────┼─────────────┤
│      M      │      5      │     29      │
└─────────────┴─────────────┴─────────────┘


In [34]:
# Compute usefull metrics between predictions and actual categories.
accuracy_test = accuracy(ŷtest,y[test])
recall_test = recall(ŷtest,y[test])
precision_test = precision(ŷtest,y[test])
print("Model Accuracy: \t $accuracy_test \nModel Recall: \t \t $recall_test \nModel Precision: \t $precision_test")

Model Accuracy: 	 0.8548387096774194 
Model Recall: 	 	 0.8787878787878788 
Model Precision: 	 0.8529411764705882

With this train-test fixed split, the SVC reachs an accuracy of 85%, an extraordinary perfomance considering there where not hyperparameter tunning. The default kernel, Radial Basis, seems to be very apprropiate to the data. In conclussion, we avoid overfitting and obtain good generalization.